# QWEN

In [ ]:
import torch
import requests
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForTokenClassification, BertTokenizerFast, BertForTokenClassification
from transformers import pipeline
from typing import List, Dict, Tuple
from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.metrics import classification_report
import numpy as np
from tqdm import tqdm
import os

# Конфигурация
DEVICE = torch.device("cpu")
QWEN_MODEL_NAME =  "Qwen/Qwen2.5-32B-Instruct"
BERT_MODEL_NAME = "Davlan/bert-base-multilingual-cased-ner-hrl" # Предобученная модель для NER

from bs4 import BeautifulSoup, Comment
import requests

def extract_text_from_url(url: str, timeout: int = 10) -> str:
    """
    Извлекает основной текстовый контент с веб-страницы по URL.

    Параметры:
        url (str): URL веб-страницы
        timeout (int): Таймаут запроса в секундах (по умолчанию 10)

    Возвращает:
        str: Очищенный текстовый контент страницы
    """
    try:
        # Заголовки для имитации браузера
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

        # Запрос с таймаутом и заголовками
        response = requests.get(url, headers=headers, timeout=timeout)
        response.raise_for_status()  # Проверка на ошибки HTTP

        soup = BeautifulSoup(response.text, 'html.parser')

        # Удаление нежелательных элементов
        for element in soup(['script', 'style', 'nav', 'footer', 'iframe', 'noscript', 'svg']):
            element.decompose()

        # Удаление HTML-комментариев
        for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
            comment.extract()

        # Извлечение текста из основных содержательных тегов
        text_elements = []
        for tag in ['article', 'main', 'div', 'section', 'p']:
            elements = soup.find_all(tag)
            for element in elements:
                # Проверка на содержание значимого текста
                if len(element.get_text(strip=True)) > 50:  # Минимум 50 символов
                    text_elements.append(element.get_text(separator=' ', strip=True))

        # Если не нашли достаточно текста в специфичных тегах, берем весь body
        if not text_elements or sum(len(t) for t in text_elements) < 500:
            text_elements = [soup.get_text(separator=' ', strip=True)]

        # Объединение и очистка текста
        text = ' '.join(text_elements)

        # Удаление лишних пробелов и переносов строк
        text = ' '.join(text.split())

        return text

    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе к {url}: {str(e)}")
        return ""
    except Exception as e:
        print(f"Неожиданная ошибка при обработке {url}: {str(e)}")
        return ""

def annotate_with_bert(text: str) -> List[Dict]:
    """Аннотирует текст с помощью предобученной модели BERT."""
    tokenizer = BertTokenizerFast.from_pretrained(BERT_MODEL_NAME)
    model = BertForTokenClassification.from_pretrained(BERT_MODEL_NAME).to(DEVICE)

    nlp = pipeline("ner", model=model, tokenizer=tokenizer, device=DEVICE)
    annotations = nlp(text)
    return annotations

def adapt_qwen_for_ner(model_name: str = QWEN_MODEL_NAME):
    """Загружает модель Qwen и адаптирует её для NER."""
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForTokenClassification.from_pretrained(
        model_name,
        num_labels=9,  # Стандартное количество классов для NER (PER, ORG, LOC и т.д.)
        id2label={0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG',
                  5: 'B-LOC', 6: 'I-LOC', 7: 'B-MISC', 8: 'I-MISC'},
        label2id={'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4,
                  'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}
    ).to(DEVICE)

    # Замораживаем все слои, кроме последнего
    for param in model.base_model.parameters():
        param.requires_grad = False

    return model, tokenizer

class NERDataset(Dataset):
    """Кастомный Dataset для NER."""
    def __init__(self, texts, annotations, tokenizer, model_config, max_length=128):
        self.texts = texts
        self.annotations = annotations
        self.tokenizer = tokenizer
        self.model_config = model_config  # Добавляем конфигурацию модели
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        annotations = self.annotations[idx]

        # Токенизация с выравниванием меток
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt',
            return_offsets_mapping=True
        )

        # Создаем массив меток, заполненный -100 (игнорируется при вычислении потерь)
        labels = torch.full((self.max_length,), -100, dtype=torch.long)

        # Преобразуем аннотации в метки для токенов
        offset_mapping = encoding['offset_mapping'][0]
        for ann in annotations:
            start, end = ann['start'], ann['end']
            label = ann['entity']

            # Находим токены, которые пересекаются с аннотацией
            for i, (token_start, token_end) in enumerate(offset_mapping):
                if token_start >= end or token_end <= start:
                    continue
                if token_start != 0 or token_end != 0:  # Игнорируем специальные токены
                    labels[i] = self.model_config.label2id.get(label, 0)  # Используем self.model_config

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': labels
        }

def prepare_training_data(text, bert_annotations, tokenizer, test_size=0.2):
    """Подготавливает данные для обучения и валидации."""
    # Разделяем текст на предложения (упрощенно)
    sentences = [sent.strip() for sent in text.split('.') if len(sent.strip()) > 0]

    # Сопоставляем аннотации с предложениями (упрощенно)
    sentence_annotations = []
    current_pos = 0
    for sent in sentences:
        sent_len = len(sent)
        sent_anns = []
        for ann in bert_annotations:
            if current_pos <= ann['start'] < current_pos + sent_len:
                adjusted_ann = {
                    'start': ann['start'] - current_pos,
                    'end': ann['end'] - current_pos,
                    'entity': ann['entity']
                }
                sent_anns.append(adjusted_ann)
        sentence_annotations.append(sent_anns)
        current_pos += sent_len + 1  # +1 для точки

    # Разделяем на train и test
    split_idx = int(len(sentences) * (1 - test_size))
    train_texts, train_anns = sentences[:split_idx], sentence_annotations[:split_idx]
    val_texts, val_anns = sentences[split_idx:], sentence_annotations[split_idx:]

    return train_texts, train_anns, val_texts, val_anns

def train_qwen(model, tokenizer, text, bert_annotations, epochs=3, batch_size=8, learning_rate=2e-5):
    """Полноценная функция обучения модели Qwen для NER."""

    # Подготовка данных
    train_texts, train_anns, val_texts, val_anns = prepare_training_data(text, bert_annotations, tokenizer)

    train_dataset = NERDataset(train_texts, train_anns, tokenizer, model_config=model.config)
    val_dataset = NERDataset(val_texts, val_anns, tokenizer, model_config=model.config)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Настройка оптимизатора и планировщика
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    total_steps = len(train_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    # Обучение
    best_val_loss = float('inf')
    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}/{epochs}")

        # Фаза обучения
        model.train()
        train_loss = 0
        for batch in tqdm(train_loader, desc="Training"):
            batch = {k: v.to(DEVICE) for k, v in batch.items()}

            optimizer.zero_grad()
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )
            loss = outputs.loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        # Фаза валидации
        model.eval()
        val_loss = 0
        all_preds = []
        all_labels = []
        for batch in tqdm(val_loader, desc="Validating"):
            batch = {k: v.to(DEVICE) for k, v in batch.items()}

            with torch.no_grad():
                outputs = model(
                    input_ids=batch['input_ids'],
                    attention_mask=batch['attention_mask'],
                    labels=batch['labels']
                )

            val_loss += outputs.loss.item()

            # Собираем предсказания и метки для метрик
            preds = torch.argmax(outputs.logits, dim=2)
            mask = batch['labels'] != -100  # Игнорируем метки -100

            all_preds.extend(preds[mask].cpu().numpy())
            all_labels.extend(batch['labels'][mask].cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)

        # Выводим метрики
        print(f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
        # print(classification_report(
        #     all_labels,
        #     all_preds,
        #     target_names=list(model.config.id2label.values()),
        #     zero_division=0
        # ))

        # Сохраняем лучшую модель
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            os.makedirs("./qwen_ner", exist_ok=True)
            model.save_pretrained("./qwen_ner")
            tokenizer.save_pretrained("./qwen_ner")
            print("Saved best model!")

    return model

def predict_with_qwen(text: str, model, tokenizer) -> List[Tuple[str, str]]:
    """Делает предсказания с помощью адаптированной модели Qwen."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(DEVICE)
    with torch.no_grad():
        outputs = model(**inputs)

    predictions = torch.argmax(outputs.logits, dim=2)
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    labels = [model.config.id2label[p.item()] for p in predictions[0]]

    return list(zip(tokens, labels))

def main(url: str):
    # 1. Извлекаем текст с веб-страницы
    text = extract_text_from_url(url)
    print(f"Извлечённый текст:\n{text[:500]}...\n")

    # 2. Аннотируем с помощью BERT
    bert_annotations = annotate_with_bert(text[:512])  # Ограничиваем длину для BERT
    print(f"Пример аннотаций BERT:\n{bert_annotations[:5]}\n")

    # 3. Адаптируем Qwen для NER
    qwen_model, qwen_tokenizer = adapt_qwen_for_ner()

    # 4. Обучаем модель
    qwen_model = train_qwen(qwen_model, qwen_tokenizer, text[:2000], bert_annotations, epochs=3)

    # 5. Делаем предсказания
    predictions = predict_with_qwen(text[:512], qwen_model, qwen_tokenizer)

    # 6. Выводим результаты
    print("Предсказания модели Qwen (первые 20 токенов):")
    for token, label in predictions[:20]:
        print(f"{token} -> {label}")

if __name__ == "__main__":
    url = "https://amulex.ru/daily/news/vozrozhdenie-lyutovolkov-iz-igry-prestolov-nauchnyj-proryv-bjfj4kf/?ysclid=mae5l3wify323318294"  # Пример URL
    main(url)

Извлечённый текст:
Блог / Новости Возрождение лютоволков из «Игры престолов»: научный прорыв Полина Недашковская 07 мая, 2025 • 2 мин • 155 Копировать ссылку Telegram VK WhatsApp Биотехнологическая компания Colossal Biosciences из Далласа совершила прорыв в области возрождения вымерших видов животных, воссоздав ужасных волков (dire wolves), известных широкой аудитории как «лютоволки» из «Игры престолов» . Процесс был долгим и сложным, но в итоге ученые смогли вернуть к жизни этот вид, вымерший более 12 тысяч лет н...



Device set to use cpu


Пример аннотаций BERT:
[{'entity': 'B-PER', 'score': np.float32(0.9994863), 'index': 26, 'word': 'Пол', 'start': 74, 'end': 77}, {'entity': 'I-PER', 'score': np.float32(0.98204434), 'index': 27, 'word': '##ина', 'start': 77, 'end': 80}, {'entity': 'I-PER', 'score': np.float32(0.9996425), 'index': 28, 'word': 'Не', 'start': 81, 'end': 83}, {'entity': 'I-PER', 'score': np.float32(0.9996587), 'index': 29, 'word': '##да', 'start': 83, 'end': 85}, {'entity': 'I-PER', 'score': np.float32(0.9995888), 'index': 30, 'word': '##шко', 'start': 85, 'end': 88}]



Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

Some weights of Qwen2ForTokenClassification were not initialized from the model checkpoint at Qwen/Qwen2.5-32B-Instruct and are newly initialized: ['score.bias', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Train Loss: nan | Val Loss: nan

Epoch 2/3


Validating: 100%|██████████| 1/1 [03:03<00:00, 183.90s/it]


Train Loss: nan | Val Loss: nan

Epoch 3/3


Validating: 100%|██████████| 1/1 [02:54<00:00, 174.30s/it]


Train Loss: nan | Val Loss: nan
Предсказания модели Qwen (первые 20 токенов):
Ðĳ -> B-PER
Ð»Ð¾Ð³ -> B-PER
Ġ/ -> B-MISC
ĠÐĿ -> B-PER
Ð¾Ð² -> B-PER
Ð¾ÑģÑĤÐ¸ -> B-PER
ĠÐĴÐ¾Ð· -> B-PER
ÑĢÐ¾Ð¶ -> B-MISC
Ð´ -> B-PER
ÐµÐ½Ð¸Ðµ -> I-PER
ĠÐ» -> B-PER
ÑİÑĤ -> B-PER
Ð¾Ð² -> B-PER
Ð¾Ð» -> B-PER
ÐºÐ¾Ð² -> B-PER
ĠÐ¸Ð· -> B-PER
ĠÂ« -> I-ORG
Ðĺ -> B-MISC
Ð³ -> B-PER
ÑĢÑĭ -> B-MISC


# SBERBANK AI - основная модель (все остальные - экспериментальные наработки)

In [46]:
import torch
import requests
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForTokenClassification, BertTokenizerFast, BertForTokenClassification
from transformers import pipeline
from typing import List, Dict, Tuple
from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.metrics import classification_report
import numpy as np
from tqdm import tqdm
import os
from bs4 import BeautifulSoup, Comment
import requests
from huggingface_hub import login



# Конфигурация
DEVICE = torch.device("cpu")
QWEN_MODEL_NAME =  "sberbank-ai/ruRoberta-large"
PUBLIC_MODELS = {
    "DeepPavlov/bert-base-cased-conversational-ner": "Русская NER (рекомендуется)",
    "Davlan/bert-base-multilingual-cased-ner-hrl": "Многоязычная NER"
}

# Автоматический выбор рабочей модели
for model_name, desc in PUBLIC_MODELS.items():
    try:
        print(f"Пробуем загрузить {model_name} ({desc})...")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForTokenClassification.from_pretrained(model_name)
        BERT_MODEL_NAME = model_name
        print(f"Успешно! Используем {model_name}")
        break
    except Exception as e:
        print(f"Ошибка загрузки {model_name}: {str(e)}")
else:
    raise ValueError("Не удалось загрузить ни одну публичную модель!")

BERT_MODEL_NAME = "Davlan/bert-base-multilingual-cased-ner-hrl"  # Предобученная BERT для NER


def is_bad_encoding(text: str) -> bool:
    return any(c in text for c in ('Ð', 'Ñ', 'â'))

def fix_encoding(text: str) -> str:
    try:
        return text.encode('latin1').decode('utf-8')
    except:
        try:
            return text.encode('iso-8859-1').decode('utf-8')
        except:
            return text

def clean_text(text: str) -> str:
    """Очистка текста от артефактов"""
    if is_bad_encoding(text):
        text = fix_encoding(text)
    return ' '.join(text.split())


TOKENIZER = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)

def extract_text_from_url(url: str, timeout: int = 10) -> str:
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=timeout)
        response.raise_for_status()

        # NEW: Принудительная установка кодировки
        response.encoding = 'utf-8'
        text = response.text

        # NEW: Проверка и исправление кодировки
        if is_bad_encoding(text):
            text = fix_encoding(text)

        soup = BeautifulSoup(text, 'html.parser')

        for element in soup(['script', 'style', 'nav', 'footer', 'iframe', 'noscript', 'svg']):
            element.decompose()

        for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
            comment.extract()

        text_elements = []
        for tag in ['article', 'main', 'div', 'section', 'p']:
            elements = soup.find_all(tag)
            for element in elements:
                if len(element.get_text(strip=True)) > 50:
                    text_elements.append(element.get_text(separator=' ', strip=True))

        if not text_elements or sum(len(t) for t in text_elements) < 500:
            text_elements = [soup.get_text(separator=' ', strip=True)]

        text = ' '.join(text_elements)
        return ' '.join(text.split())

    except Exception as e:
        print(f"Ошибка при обработке URL: {str(e)}")
        return ""

def annotate_with_bert(text: str) -> Tuple[List[Dict], Dict[str, int]]:
    try:
        tokenizer = BertTokenizerFast.from_pretrained(
            BERT_MODEL_NAME,
            do_lower_case=False,
            strip_accents=False
        )
        model = AutoModelForTokenClassification.from_pretrained(BERT_MODEL_NAME).to(DEVICE)

        nlp = pipeline("ner", model=model, tokenizer=TOKENIZER,
                      device=DEVICE, aggregation_strategy=None)  # Убрали агрегацию

        raw_annotations = nlp(text[:512])

        # Конвертация в BIO-формат с сохранением I-меток
        annotations = []
        for i, ann in enumerate(raw_annotations):
            entity = ann['entity']
            # Обработка частей слов
            if ann['word'].startswith('##'):
                if annotations:
                    annotations[-1]['word'] += ann['word'][2:]
                    annotations[-1]['end'] = ann['end']
                continue

            new_ann = {
                'start': ann['start'],
                'end': ann['end'],
                'entity': entity,  # Сохраняем оригинальные метки (уже в BIO)
                'word': ann['word'],
                'score': ann['score']
            }
            annotations.append(new_ann)

        # Статистика по типам сущностей
        type_counts = {'PER': 0, 'ORG': 0, 'LOC': 0, 'MISC': 0}
        for ann in annotations:
            if ann['entity'] != 'O':
                ent_type = ann['entity'].split('-')[-1]
                type_counts[ent_type] = type_counts.get(ent_type, 0) + 1

        return annotations, type_counts

    except Exception as e:
        print(f"Ошибка аннотации: {str(e)}")
        return [], {'PER': 0, 'ORG': 0, 'LOC': 0, 'MISC': 0}

def adapt_qwen_for_ner(model_name: str = QWEN_MODEL_NAME):
    """Адаптирует русский BERT для NER."""
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)

        # Явно задаем метки BIO-формата
        label2id = {
            'O': 0,
            'B-PER': 1, 'I-PER': 2,
            'B-ORG': 3, 'I-ORG': 4,
            'B-LOC': 5, 'I-LOC': 6,
            'B-MISC': 7, 'I-MISC': 8
        }

        model = AutoModelForTokenClassification.from_pretrained(
            model_name,
            num_labels=len(label2id),
            id2label={i: l for l, i in label2id.items()},
            label2id=label2id
        ).to(DEVICE)

        # Замораживаем все слои, кроме последнего
        for param in model.base_model.parameters():
            param.requires_grad = False

        return model, tokenizer

    except Exception as e:
        print(f"Ошибка при загрузке модели: {str(e)}")
        return None, None  # Возвращаем явные None при ошибке

def analyze_class_balance(annotations: List[Dict]) -> Dict[str, int]:
    """Анализ распределения меток с гарантированным возвратом всех типов"""
    bio_labels = ['B-PER', 'I-PER', 'B-ORG', 'I-ORG',
                 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC', 'O']
    counts = {label: 0 for label in bio_labels}

    for ann in annotations:
        label = ann.get('entity', 'O')
        if label in counts:
            counts[label] += 1
        else:
            # Конвертируем неизвестные метки
            if label.startswith('B-'):
                counts['B-MISC'] += 1
            elif label.startswith('I-'):
                counts['I-MISC'] += 1
            else:
                counts['O'] += 1

    return counts

class NERDataset(Dataset):
    """Кастомный Dataset для NER."""
    def __init__(self, texts, annotations, tokenizer, model_config, max_length=128):
        self.texts = texts
        self.annotations = annotations
        self.tokenizer = tokenizer
        self.model_config = model_config  # Добавляем конфигурацию модели
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        annotations = self.annotations[idx]

        # Токенизация с выравниванием меток
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt',
            return_offsets_mapping=True
        )

        # Создаем массив меток, заполненный -100 (игнорируется при вычислении потерь)
        labels = torch.full((self.max_length,), -100, dtype=torch.long)

        # Преобразуем аннотации в метки для токенов
        offset_mapping = encoding['offset_mapping'][0]
        for ann in annotations:
            start, end = ann['start'], ann['end']
            label = ann['entity']

            # Находим токены, которые пересекаются с аннотацией
            for i, (token_start, token_end) in enumerate(offset_mapping):
                if token_start >= end or token_end <= start:
                    continue
                if token_start != 0 or token_end != 0:  # Игнорируем специальные токены
                    labels[i] = self.model_config.label2id.get(label, 0)  # Используем self.model_config

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': labels
        }

def prepare_training_data(text, bert_annotations, tokenizer, test_size=0.2):
    """Подготавливает данные для обучения и валидации."""
    # Разделяем текст на предложения (упрощенно)
    sentences = [sent.strip() for sent in text.split('.') if len(sent.strip()) > 0]

    # Модифицируем аннотации для BIO-формата
    for ann in bert_annotations:
        if isinstance(ann, dict) and 'entity_group' in ann:
            ann['entity'] = f"B-{ann['entity_group']}"  # Конвертируем в BIO-формат
        elif isinstance(ann, dict):
            ann['entity'] = ann.get('entity', 'O')  # Добавляем поле entity, если его нет

    # Сопоставляем аннотации с предложениями
    sentence_annotations = []
    current_pos = 0
    for sent in sentences:
        sent_len = len(sent)
        sent_anns = []
        for ann in bert_annotations:
            if isinstance(ann, dict) and 'start' in ann and 'end' in ann:
                if current_pos <= ann['start'] < current_pos + sent_len:
                    adjusted_ann = {
                        'start': ann['start'] - current_pos,
                        'end': ann['end'] - current_pos,
                        'entity': ann['entity']  # Используем уже преобразованную метку
                    }
                    sent_anns.append(adjusted_ann)
        sentence_annotations.append(sent_anns)
        current_pos += sent_len + 1

    # Анализ баланса классов
    all_annotations = [ann for sent_anns in sentence_annotations for ann in sent_anns]
    class_counts = analyze_class_balance(all_annotations)

    print("\n=== Распределение классов ===")
    for cls, count in class_counts.items():
        print(f"{cls}: {count} ({count/len(all_annotations):.1%})" if len(all_annotations) > 0 else f"{cls}: 0")

    # Разделяем на train и test
    split_idx = int(len(sentences) * (1 - test_size))
    train_texts, train_anns = sentences[:split_idx], sentence_annotations[:split_idx]
    val_texts, val_anns = sentences[split_idx:], sentence_annotations[split_idx:]

    return train_texts, train_anns, val_texts, val_anns

def train_qwen(model, tokenizer, text, bert_annotations, epochs=10, batch_size=32, learning_rate=2e-5):
    """Полноценная функция обучения модели sberbank ai для NER."""

    # Подготовка данных
    train_texts, train_anns, val_texts, val_anns = prepare_training_data(text, bert_annotations, tokenizer)

    # Создание взвешенной функции потерь
    all_annotations = [ann for sent_anns in train_anns + val_anns for ann in sent_anns]
    class_counts = analyze_class_balance(all_annotations)

    # Создаем веса для ВСЕХ 9 классов
    class_weights = torch.ones(9, device=DEVICE) * 0.1  # Начальное значение 0.1 для всех

    # Обновляем веса для конкретных классов (кроме 'O')
    label2id = model.config.label2id
    for cls, count in class_counts.items():
        if cls != 'O' and count > 0:
            class_weights[label2id[cls]] = 1.0 / (count + 1e-5)

    criterion = torch.nn.CrossEntropyLoss(weight=class_weights, ignore_index=-100)

    train_dataset = NERDataset(train_texts, train_anns, tokenizer, model_config=model.config)
    val_dataset = NERDataset(val_texts, val_anns, tokenizer, model_config=model.config)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Настройка оптимизатора и планировщика
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    total_steps = len(train_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    # Обучение
    best_val_loss = float('inf')
    for epoch in range(epochs):
        # ... остальной код обучения ...
        print(f"\nEpoch {epoch + 1}/{epochs}")

        # Фаза обучения
        model.train()
        train_loss = 0
        for batch in tqdm(train_loader, desc="Training"):
            batch = {k: v.to(DEVICE) for k, v in batch.items()}

            optimizer.zero_grad()
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )

            # Используем нашу кастомную функцию потерь
            loss = criterion(outputs.logits.view(-1, outputs.logits.shape[-1]),
                            batch['labels'].view(-1))

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        # Фаза валидации
        model.eval()
        val_loss = 0
        all_preds = []
        all_labels = []
        for batch in tqdm(val_loader, desc="Validating"):
            batch = {k: v.to(DEVICE) for k, v in batch.items()}

            with torch.no_grad():
                outputs = model(
                    input_ids=batch['input_ids'],
                    attention_mask=batch['attention_mask'],
                    labels=batch['labels']
                )

            val_loss += outputs.loss.item()

            # Собираем предсказания и метки для метрик
            preds = torch.argmax(outputs.logits, dim=2)
            mask = batch['labels'] != -100  # Игнорируем метки -100

            all_preds.extend(preds[mask].cpu().numpy())
            all_labels.extend(batch['labels'][mask].cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)

        # Выводим метрики
        print(f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

        # Улучшенный вывод classification_report
        if len(all_labels) > 0 and len(all_preds) > 0:
            try:
                # Получаем список всех возможных классов
                unique_labels = set(all_labels) | set(all_preds)
                labels_list = sorted(list(unique_labels))

                # Фильтруем target_names по существующим классам
                existing_target_names = [name for i, name in enumerate(model.config.id2label.values())
                                       if i in unique_labels or i in labels_list]

                print(classification_report(
                    all_labels,
                    all_preds,
                    labels=labels_list,
                    target_names=existing_target_names,
                    zero_division=0
                ))
            except Exception as e:
                print(f"Ошибка при выводе метрик: {str(e)}")
        else:
            print("Нет данных для оценки метрик (возможно, все метки - 'O')")

        # Сохраняем лучшую модель
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            os.makedirs("./qwen_ner", exist_ok=True)
            model.save_pretrained("./qwen_ner")
            tokenizer.save_pretrained("./qwen_ner")
            print("Saved best model!")

    return model


def fix_encoding(text: str) -> str:
    try:
        return text.encode('latin1').decode('utf-8')
    except:
        return text

def predict_with_qwen(text: str, model, tokenizer) -> List[Tuple[str, str]]:
    # Предварительная очистка текста
    text = clean_text(text)

    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=512,
        padding='max_length'
    ).to(DEVICE)

    with torch.no_grad():
        outputs = model(**inputs)

    predictions = torch.argmax(outputs.logits, dim=2)[0]
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

    results = []
    current_word = []
    current_label = "O"

    for token, pred_idx in zip(tokens, predictions):
        if token in [tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token]:
            continue

        label = model.config.id2label.get(pred_idx.item(), "O")

        if token.startswith('##'):
            current_word.append(token[2:])
        else:
            if current_word:
                word = ''.join(current_word)
                results.append((word, current_label))
            current_word = [token]
            current_label = label

    if current_word:
        word = ''.join(current_word)
        results.append((word, current_label))

    # NEW: Постобработка результатов
    results = [(word.replace('Ġ', '').replace('▁', ''), label)
              for word, label in results]

    # NEW: Валидация BIO-разметки
    prev_label = None
    clean_results = []
    for word, label in results:
        if label.startswith('I-') and not (
            prev_label == label or
            prev_label == label.replace('I-', 'B-')
        ):
            label = 'O'  # Исправляем некорректную I-метку
        clean_results.append((word, label))
        prev_label = label if label != 'O' else None

    return clean_results


def print_predictions(predictions: List[Tuple[str, str]]):
    """Красивый вывод предсказаний"""
    from termcolor import colored

    print("\n=== Результаты распознавания ===")
    for word, label in predictions:
        if label == 'O':
            print(word, end=' ')
        else:
            color = {
                'B-PER': 'green',
                'I-PER': 'green',
                'B-ORG': 'blue',
                'I-ORG': 'blue',
                'B-LOC': 'red',
                'I-LOC': 'red'
            }.get(label, 'yellow')
            print(colored(word, color), end=' ')
    print("\n")


def main(url: str):
    # 1. Извлекаем текст
    text = extract_text_from_url(url)
    if not text:
        print("Не удалось извлечь текст")
        return

    # NEW: Дополнительная очистка текста
    text = clean_text(text)
    print("\n=== Исходный текст ===\n", text[:500], "\n")

    # 2. Аннотируем с помощью BERT
    bert_annotations, stats = annotate_with_bert(text[:512])

    if not bert_annotations:
        print("Не найдено сущностей для обучения")
        return

    # 3. Проверяем аннотации
    if not bert_annotations:
        print("Не найдено сущностей для обучения")
        return

    print("\n=== Примеры аннотаций ===")
    for i, ann in enumerate(bert_annotations[:5]):
        print(f"{ann['word']} -> {ann['entity']} (score: {ann['score']:.2f})")

    # 4. Анализ распределения
    print("\n=== Распределение сущностей ===")
    for ent_type, count in stats.items():
        print(f"{ent_type}: {count}")

    bio_counts = analyze_class_balance(bert_annotations)
    print("\n=== BIO-распределение ===")
    for label in ['B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']:
        print(f"{label}: {bio_counts[label]}")

    # 5. Адаптация и обучение модели
    qwen_model, qwen_tokenizer = adapt_qwen_for_ner()
    if qwen_model is None:
        return

    try:
        qwen_model = train_qwen(qwen_model, qwen_tokenizer,
                              text[:2000], bert_annotations, epochs=5)
    except Exception as e:
        print(f"Ошибка обучения: {e}")
        return

    # 6. Делаем предсказания
    text_sample = text[:512]  # Используем текст напрямую без нормализации
    if not text_sample:
        print("Текст пуст")
        return

    # Предсказания
    predictions = predict_with_qwen(text_sample, qwen_model, qwen_tokenizer)

    # Вывод результатов
    print_predictions(predictions)

    # Детализированный вывод
    print("\n=== Детализация сущностей ===")
    for word, label in predictions:
        if label != 'O':
            print(f"{word} -> {label}")

if __name__ == "__main__":
    url = "https://ru.wikipedia.org/wiki/История_Google"  # Пример URL
    main(url)

Пробуем загрузить DeepPavlov/bert-base-cased-conversational-ner (Русская NER (рекомендуется))...
Ошибка загрузки DeepPavlov/bert-base-cased-conversational-ner: DeepPavlov/bert-base-cased-conversational-ner is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`
Пробуем загрузить Davlan/bert-base-multilingual-cased-ner-hrl (Многоязычная NER)...
Успешно! Используем Davlan/bert-base-multilingual-cased-ner-hrl

=== Исходный текст ===
 История Google Материал из Википедии — свободной энциклопедии Перейти к навигации Перейти к поиску Компания Google официально была создана в 1998 году. Содержание 1 1996—2001 1.1 Создание поиска 1.2 Создание компании 2 2002—2015 3 2015—2019 4 См. также 5 Примечания 6 Ссылки 1996—2001 [ править | править код ] Создание поиска [ править | править

Device set to use cpu



=== Примеры аннотаций ===
Google -> B-ORG (score: 0.99)
Сергей -> B-PER (score: 1.00)
Брин -> I-PER (score: 1.00)
Google -> B-ORG (score: 1.00)
Ларри -> B-PER (score: 1.00)

=== Распределение сущностей ===
PER: 6
ORG: 4
LOC: 1
MISC: 0

=== BIO-распределение ===
B-PER: 3
I-PER: 3
B-ORG: 3
I-ORG: 1
B-LOC: 1
I-LOC: 0


Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at sberbank-ai/ruRoberta-large and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



=== Распределение классов ===
B-PER: 3 (27.3%)
I-PER: 3 (27.3%)
B-ORG: 3 (27.3%)
I-ORG: 1 (9.1%)
B-LOC: 1 (9.1%)
I-LOC: 0 (0.0%)
B-MISC: 0 (0.0%)
I-MISC: 0 (0.0%)
O: 0 (0.0%)

Epoch 1/5


...

Train Loss: 2.1666 | Val Loss: nan
Нет данных для оценки метрик (возможно, все метки - 'O')

Epoch 4/5


Validating: 100%|██████████| 1/1 [00:05<00:00,  5.67s/it]


Train Loss: 2.1931 | Val Loss: nan
Нет данных для оценки метрик (возможно, все метки - 'O')

Epoch 5/5


Validating: 100%|██████████| 1/1 [00:04<00:00,  4.98s/it]


Train Loss: 2.3689 | Val Loss: nan
Нет данных для оценки метрик (возможно, все метки - 'O')

=== Результаты распознавания ===
ÐĺÑģÑĤÐ¾ÑĢÐ¸Ñı Google ÐľÐ°ÑĤÐµÑĢÐ¸ Ð°Ð» Ð¸Ð· ÐĴÐ¸ÐºÐ¸ Ð¿ÐµÐ´ Ð¸Ð¸ âĢĶ ÑģÐ²Ð¾Ð±Ð¾Ð´Ð½Ð¾Ð¹ ÑįÐ½ÑĨÐ¸ÐºÐ»Ð¾Ð¿ÐµÐ´ Ð¸Ð¸ ÐŁÐµÑĢ ÐµÐ¹ ÑĤÐ¸ Ðº Ð½Ð°Ð²Ð¸Ð³ Ð°ÑĨÐ¸Ð¸ ÐŁÐµÑĢ ÐµÐ¹ ÑĤÐ¸ Ðº Ð¿Ð¾Ð¸ÑģÐº Ñĥ ÐļÐ¾Ð¼Ð¿Ð°Ð½Ð¸Ñı Google Ð¾ÑĦÐ¸ÑĨÐ¸Ð°Ð»ÑĮÐ½Ð¾ Ð±ÑĭÐ»Ð° ÑģÐ¾Ð·Ð´Ð°Ð½Ð° Ð² 1998 Ð³Ð¾Ð´Ñĥ . Ð¡Ð¾Ð´ÐµÑĢÐ¶ Ð°Ð½Ð¸Ðµ 1 1996 âĢĶ 2001 1 . 1 Ð¡Ð¾Ð·Ð´ Ð°Ð½Ð¸Ðµ Ð¿Ð¾Ð¸ÑģÐºÐ° 1 . 2 Ð¡Ð¾Ð·Ð´ Ð°Ð½Ð¸Ðµ ÐºÐ¾Ð¼Ð¿Ð°Ð½Ð¸Ð¸ 2 2002 âĢĶ 2015 3 2015 âĢĶ 20 19 4 Ð¡Ð¼ . ÑĤÐ°ÐºÐ¶Ðµ 5 ÐŁÑĢÐ¸Ð¼ÐµÑĩ Ð°Ð½Ð¸Ñı 6 Ð¡ÑģÑĭÐ»ÐºÐ¸ 1996 âĢĶ 2001 [ Ð¿ÑĢÐ°Ð²Ð¸ÑĤÑĮ | Ð¿ÑĢÐ°Ð²Ð¸ÑĤÑĮ ÐºÐ¾Ð´  ] Ð¡Ð¾Ð·Ð´ Ð°Ð½Ð¸Ðµ Ð¿Ð¾Ð¸ÑģÐºÐ° [ Ð¿ÑĢÐ°Ð²Ð¸ÑĤÑĮ | Ð¿ÑĢÐ°Ð²Ð¸ÑĤÑĮ ÐºÐ¾Ð´  ] Ð¡ÐµÑĢÐ³ÐµÐ¹ ÐĳÑĢ Ð¸Ð½ ÐļÐ¾Ð¼Ð¿Ð°Ð½Ð¸Ñı Google Ð¿Ð¾ÑıÐ²Ð¸Ð»Ð°ÑģÑĮ ÐºÐ°Ðº ÑĢÐ°Ð·Ð²Ð¸ÑĤÐ¸Ðµ Ð½Ð°ÑĥÑĩÐ½Ð¾Ð³Ð¾ Ð¿ÑĢÐ¾ÐµÐºÑĤÐ° ÐĽÐ°ÑĢ ÑĢÐ¸ ÐŁÐµÐ¹ Ð´Ð¶Ð° Ð¸ Ð¡ÐµÑĢÐ³ÐµÑı ÐĳÑĢ Ð¸Ð½Ð° . ÐĴ 1996 Ð³Ð¾Ð´Ñĥ ÑģÑĤÑĥÐ´ÐµÐ½ÑĤÑĭ Ð¡ÑĤÑįÐ½ ÑĦÐ¾ÑĢÐ´Ð° ÑĢÐ°Ð±Ð¾ÑĤÐ°Ð»Ð¸ Ð½Ð°Ð´ 

In [38]:
pip install ftfy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.5 MB/s eta 0:00:00


# FACEBOOK AI

In [ ]:
import torch
import requests
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from typing import List, Dict, Tuple
from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.metrics import classification_report
import numpy as np
from tqdm import tqdm
import os

# Конфигурация
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_NAME = "FacebookAI/xlm-roberta-large-finetuned-conll03-english"

from bs4 import BeautifulSoup, Comment
import requests

def extract_text_from_url(url: str, timeout: int = 10) -> str:
    """
    Извлекает основной текстовый контент с веб-страницы по URL.

    Параметры:
        url (str): URL веб-страницы
        timeout (int): Таймаут запроса в секундах (по умолчанию 10)

    Возвращает:
        str: Очищенный текстовый контент страницы
    """
    try:
        # Заголовки для имитации браузера
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

        # Запрос с таймаутом и заголовками
        response = requests.get(url, headers=headers, timeout=timeout)
        response.raise_for_status()  # Проверка на ошибки HTTP

        soup = BeautifulSoup(response.text, 'html.parser')

        # Удаление нежелательных элементов
        for element in soup(['script', 'style', 'nav', 'footer', 'iframe', 'noscript', 'svg']):
            element.decompose()

        # Удаление HTML-комментариев
        for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
            comment.extract()

        # Извлечение текста из основных содержательных тегов
        text_elements = []
        for tag in ['article', 'main', 'div', 'section', 'p']:
            elements = soup.find_all(tag)
            for element in elements:
                # Проверка на содержание значимого текста
                if len(element.get_text(strip=True)) > 50:  # Минимум 50 символов
                    text_elements.append(element.get_text(separator=' ', strip=True))

        # Если не нашли достаточно текста в специфичных тегах, берем весь body
        if not text_elements or sum(len(t) for t in text_elements) < 500:
            text_elements = [soup.get_text(separator=' ', strip=True)]

        # Объединение и очистка текста
        text = ' '.join(text_elements)

        # Удаление лишних пробелов и переносов строк
        text = ' '.join(text.split())

        return text

    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе к {url}: {str(e)}")
        return ""
    except Exception as e:
        print(f"Неожиданная ошибка при обработке {url}: {str(e)}")
        return ""

def annotate_with_xlm_roberta(text: str) -> List[Dict]:
    """Аннотирует текст с помощью предобученной модели XLM-RoBERTa."""
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME).to(DEVICE)

    nlp = pipeline("ner", model=model, tokenizer=tokenizer, device=0 if DEVICE.type == "cuda" else -1)
    annotations = nlp(text)

    # Добавляем индекс сущности в соответствии с id2label модели
    label2id = model.config.label2id
    for ann in annotations:
        ann['index'] = label2id[ann['entity']]

    return annotations

class NERDataset(Dataset):
    """Кастомный Dataset для NER."""
    def __init__(self, texts, annotations, tokenizer, max_length=128):
        self.texts = texts
        self.annotations = annotations
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.label_map = {
            'O': 0,
            'B-PER': 1,
            'I-PER': 2,
            'B-ORG': 3,
            'I-ORG': 4,
            'B-LOC': 5,
            'I-LOC': 6,
            'B-MISC': 7,
            'I-MISC': 8
        }

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        annotations = self.annotations[idx]

        # Токенизация с выравниванием меток
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt',
            return_offsets_mapping=True
        )

        # Создаем массив меток, заполненный -100 (игнорируется при вычислении потерь)
        labels = torch.full((self.max_length,), -100, dtype=torch.long)

        # Преобразуем аннотации в метки для токенов
        offset_mapping = encoding['offset_mapping'][0]
        for ann in annotations:
            start, end = ann['start'], ann['end']
            label_idx = ann['index']  # Используем предварительно сохраненный индекс

            # Находим токены, которые пересекаются с аннотацией
            for i, (token_start, token_end) in enumerate(offset_mapping):
                if token_start >= end or token_end <= start:
                    continue
                if token_start != 0 or token_end != 0:  # Игнорируем специальные токены
                    # Убедимся, что индекс метки в допустимых пределах
                    if 0 <= label_idx < len(self.label_map):
                        labels[i] = label_idx
                    else:
                        labels[i] = 0  # По умолчанию 'O'

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': labels
        }

def prepare_training_data(text, annotations, tokenizer, test_size=0.2):
    """Подготавливает данные для обучения и валидации."""
    # Разделяем текст на предложения (упрощенно)
    sentences = [sent.strip() for sent in text.split('.') if len(sent.strip()) > 0]

    # Сопоставляем аннотации с предложениями (упрощенно)
    sentence_annotations = []
    current_pos = 0
    for sent in sentences:
        sent_len = len(sent)
        sent_anns = []
        for ann in annotations:
            if current_pos <= ann['start'] < current_pos + sent_len:
                adjusted_ann = {
                    'start': ann['start'] - current_pos,
                    'end': ann['end'] - current_pos,
                    'entity': ann['entity'],
                    'index': ann['index']  # Сохраняем индекс сущности
                }
                sent_anns.append(adjusted_ann)
        sentence_annotations.append(sent_anns)
        current_pos += sent_len + 1  # +1 для точки

    # Разделяем на train и test
    split_idx = int(len(sentences) * (1 - test_size))
    train_texts, train_anns = sentences[:split_idx], sentence_annotations[:split_idx]
    val_texts, val_anns = sentences[split_idx:], sentence_annotations[split_idx:]

    return train_texts, train_anns, val_texts, val_anns

def train_model(model, tokenizer, text, annotations, epochs=3, batch_size=8, learning_rate=2e-5):
    """Функция обучения модели XLM-RoBERTa для NER."""

    # Подготовка данных
    train_texts, train_anns, val_texts, val_anns = prepare_training_data(text, annotations, tokenizer)

    train_dataset = NERDataset(train_texts, train_anns, tokenizer)
    val_dataset = NERDataset(val_texts, val_anns, tokenizer)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Настройка оптимизатора и планировщика
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    total_steps = len(train_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    # Обучение
    best_val_loss = float('inf')
    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}/{epochs}")

        # Фаза обучения
        model.train()
        train_loss = 0
        for batch in tqdm(train_loader, desc="Training"):
            batch = {k: v.to(DEVICE) for k, v in batch.items()}

            optimizer.zero_grad()
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )
            loss = outputs.loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        # Фаза валидации
        model.eval()
        val_loss = 0
        all_preds = []
        all_labels = []
        for batch in tqdm(val_loader, desc="Validating"):
            batch = {k: v.to(DEVICE) for k, v in batch.items()}

            with torch.no_grad():
                outputs = model(
                    input_ids=batch['input_ids'],
                    attention_mask=batch['attention_mask'],
                    labels=batch['labels']
                )

            val_loss += outputs.loss.item()

            # Собираем предсказания и метки для метрик
            preds = torch.argmax(outputs.logits, dim=2)
            mask = batch['labels'] != -100  # Игнорируем метки -100

            all_preds.extend(preds[mask].cpu().numpy())
            all_labels.extend(batch['labels'][mask].cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)

        # Выводим метрики
        print(f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
        # print(classification_report(
        #     all_labels,
        #     all_preds,
        #     target_names=list(model.config.id2label.values()),
        #     zero_division=0
        # ))

        # Сохраняем лучшую модель
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            os.makedirs("./xlm_roberta_ner", exist_ok=True)
            model.save_pretrained("./xlm_roberta_ner")
            tokenizer.save_pretrained("./xlm_roberta_ner")
            print("Saved best model!")

    return model

def predict_with_model(text: str, model, tokenizer) -> List[Tuple[str, str]]:
    """Делает предсказания с помощью модели XLM-RoBERTa."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(DEVICE)
    with torch.no_grad():
        outputs = model(**inputs)

    predictions = torch.argmax(outputs.logits, dim=2)
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    labels = [model.config.id2label[p.item()] for p in predictions[0]]

    return list(zip(tokens, labels))

def main(url: str):
    # 1. Извлекаем текст с веб-страницы
    text = extract_text_from_url(url)
    print(f"Извлечённый текст:\n{text[:500]}...\n")

    # 2. Аннотируем с помощью XLM-RoBERTa
    annotations = annotate_with_xlm_roberta(text[:512])  # Ограничиваем длину для модели
    print(f"Пример аннотаций XLM-RoBERTa:\n{annotations[:5]}\n")

    # 3. Загружаем модель и токенизатор
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME).to(DEVICE)

    # 4. Обучаем модель (дообучение)
    model = train_model(model, tokenizer, text[:2000], annotations, epochs=3)

    # 5. Делаем предсказания
    predictions = predict_with_model(text[:512], model, tokenizer)

    # 6. Выводим результаты
    print("Предсказания модели XLM-RoBERTa (первые 20 токенов):")
    for token, label in predictions[:20]:
        print(f"{token} -> {label}")

if __name__ == "__main__":
    url = "https://amulex.ru/daily/news/vozrozhdenie-lyutovolkov-iz-igry-prestolov-nauchnyj-proryv-bjfj4kf/?ysclid=mae5l3wify323318294"
    main(url)

Извлечённый текст:
Блог / Новости Возрождение лютоволков из «Игры престолов»: научный прорыв Полина Недашковская 07 мая, 2025 • 2 мин • 144 Копировать ссылку Telegram VK WhatsApp Биотехнологическая компания Colossal Biosciences из Далласа совершила прорыв в области возрождения вымерших видов животных, воссоздав ужасных волков (dire wolves), известных широкой аудитории как «лютоволки» из «Игры престолов» . Процесс был долгим и сложным, но в итоге ученые смогли вернуть к жизни этот вид, вымерший более 12 тысяч лет н...



Some weights of the model checkpoint at FacebookAI/xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Пример аннотаций XLM-RoBERTa:
[{'entity': 'I-MISC', 'score': np.float32(0.9999461), 'index': 4, 'word': 'И', 'start': 42, 'end': 43}, {'entity': 'I-MISC', 'score': np.float32(0.9999021), 'index': 4, 'word': 'г', 'start': 43, 'end': 44}, {'entity': 'I-MISC', 'score': np.float32(0.99977), 'index': 4, 'word': 'ры', 'start': 44, 'end': 46}, {'entity': 'I-MISC', 'score': np.float32(0.99995315), 'index': 4, 'word': '▁престол', 'start': 47, 'end': 54}, {'entity': 'I-MISC', 'score': np.float32(0.99939775), 'index': 4, 'word': 'ов', 'start': 54, 'end': 56}]



Some weights of the model checkpoint at FacebookAI/xlm-roberta-large-finetuned-conll03-english were not used when initializing XLMRobertaForTokenClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Epoch 1/3


...

Train Loss: nan | Val Loss: nan

Epoch 2/3


Validating: 100%|██████████| 1/1 [00:04<00:00,  4.99s/it]


Train Loss: nan | Val Loss: nan

Epoch 3/3


Validating: 100%|██████████| 1/1 [00:05<00:00,  5.29s/it]


Train Loss: nan | Val Loss: nan
Предсказания модели XLM-RoBERTa (первые 20 токенов):
<s> -> O
▁Блог -> O
▁/ -> O
▁Новости -> O
▁Воз -> O
рожден -> O
ие -> O
▁лют -> O
о -> O
вол -> O
ков -> O
▁из -> O
▁« -> O
И -> I-MISC
г -> I-MISC
ры -> I-MISC
▁престол -> I-MISC
ов -> I-MISC
»: -> O
▁на -> O


⁉
# Babelscape/wikineural-multilingual-ner
---



In [ ]:
import torch
import requests
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from typing import List, Dict, Tuple
from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.metrics import classification_report
import numpy as np
from tqdm import tqdm
import os

# Конфигурация
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_NAME = "Babelscape/wikineural-multilingual-ner"

from bs4 import BeautifulSoup, Comment
import requests

def extract_text_from_url(url: str, timeout: int = 10) -> str:
    """
    Извлекает основной текстовый контент с веб-страницы по URL.

    Параметры:
        url (str): URL веб-страницы
        timeout (int): Таймаут запроса в секундах (по умолчанию 10)

    Возвращает:
        str: Очищенный текстовый контент страницы
    """
    try:
        # Заголовки для имитации браузера
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

        # Запрос с таймаутом и заголовками
        response = requests.get(url, headers=headers, timeout=timeout)
        response.raise_for_status()  # Проверка на ошибки HTTP

        soup = BeautifulSoup(response.text, 'html.parser')

        # Удаление нежелательных элементов
        for element in soup(['script', 'style', 'nav', 'footer', 'iframe', 'noscript', 'svg']):
            element.decompose()

        # Удаление HTML-комментариев
        for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
            comment.extract()

        # Извлечение текста из основных содержательных тегов
        text_elements = []
        for tag in ['article', 'main', 'div', 'section', 'p']:
            elements = soup.find_all(tag)
            for element in elements:
                # Проверка на содержание значимого текста
                if len(element.get_text(strip=True)) > 50:  # Минимум 50 символов
                    text_elements.append(element.get_text(separator=' ', strip=True))

        # Если не нашли достаточно текста в специфичных тегах, берем весь body
        if not text_elements or sum(len(t) for t in text_elements) < 500:
            text_elements = [soup.get_text(separator=' ', strip=True)]

        # Объединение и очистка текста
        text = ' '.join(text_elements)

        # Удаление лишних пробелов и переносов строк
        text = ' '.join(text.split())

        return text

    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе к {url}: {str(e)}")
        return ""
    except Exception as e:
        print(f"Неожиданная ошибка при обработке {url}: {str(e)}")
        return ""

def annotate_with_wikineural(text: str) -> List[Dict]:
    """Аннотирует текст с помощью модели WikiNeural."""
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME).to(DEVICE)

    nlp = pipeline("ner", model=model, tokenizer=tokenizer, device=0 if DEVICE.type == "cuda" else -1)
    annotations = nlp(text)

    # Добавляем индекс сущности в соответствии с id2label модели
    label2id = model.config.label2id
    for ann in annotations:
        ann['index'] = label2id[ann['entity']]

    return annotations

class NERDataset(Dataset):
    """Кастомный Dataset для NER."""
    def __init__(self, texts, annotations, tokenizer, max_length=128):
        self.texts = texts
        self.annotations = annotations
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        annotations = self.annotations[idx]

        # Токенизация с выравниванием меток
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            truncation=True,
            padding='max_length',
            return_tensors='pt',
            return_offsets_mapping=True
        )

        # Создаем массив меток, заполненный -100 (игнорируется при вычислении потерь)
        labels = torch.full((self.max_length,), -100, dtype=torch.long)

        # Преобразуем аннотации в метки для токенов
        offset_mapping = encoding['offset_mapping'][0]
        for ann in annotations:
            start, end = ann['start'], ann['end']
            label_idx = ann['index']

            # Находим токены, которые пересекаются с аннотацией
            for i, (token_start, token_end) in enumerate(offset_mapping):
                if token_start >= end or token_end <= start:
                    continue
                if token_start != 0 or token_end != 0:  # Игнорируем специальные токены
                    labels[i] = label_idx

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': labels
        }

def prepare_training_data(text, annotations, tokenizer, test_size=0.2):
    """Подготавливает данные для обучения и валидации."""
    # Разделяем текст на предложения (упрощенно)
    sentences = [sent.strip() for sent in text.split('.') if len(sent.strip()) > 0]

    # Сопоставляем аннотации с предложениями (упрощенно)
    sentence_annotations = []
    current_pos = 0
    for sent in sentences:
        sent_len = len(sent)
        sent_anns = []
        for ann in annotations:
            if current_pos <= ann['start'] < current_pos + sent_len:
                adjusted_ann = {
                    'start': ann['start'] - current_pos,
                    'end': ann['end'] - current_pos,
                    'entity': ann['entity'],
                    'index': ann['index']
                }
                sent_anns.append(adjusted_ann)
        sentence_annotations.append(sent_anns)
        current_pos += sent_len + 1  # +1 для точки

    # Разделяем на train и test
    split_idx = int(len(sentences) * (1 - test_size))
    train_texts, train_anns = sentences[:split_idx], sentence_annotations[:split_idx]
    val_texts, val_anns = sentences[split_idx:], sentence_annotations[split_idx:]

    return train_texts, train_anns, val_texts, val_anns

def train_model(model, tokenizer, text, annotations, epochs=3, batch_size=8, learning_rate=2e-5):
    """Функция обучения модели WikiNeural для NER."""

    # Подготовка данных
    train_texts, train_anns, val_texts, val_anns = prepare_training_data(text, annotations, tokenizer)

    train_dataset = NERDataset(train_texts, train_anns, tokenizer)
    val_dataset = NERDataset(val_texts, val_anns, tokenizer)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Настройка оптимизатора и планировщика
    optimizer = AdamW(model.parameters(), lr=learning_rate)
    total_steps = len(train_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    # Обучение
    best_val_loss = float('inf')
    for epoch in range(epochs):
        print(f"\nEpoch {epoch + 1}/{epochs}")

        # Фаза обучения
        model.train()
        train_loss = 0
        for batch in tqdm(train_loader, desc="Training"):
            batch = {k: v.to(DEVICE) for k, v in batch.items()}

            optimizer.zero_grad()
            outputs = model(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels']
            )
            loss = outputs.loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        # Фаза валидации
        model.eval()
        val_loss = 0
        all_preds = []
        all_labels = []
        for batch in tqdm(val_loader, desc="Validating"):
            batch = {k: v.to(DEVICE) for k, v in batch.items()}

            with torch.no_grad():
                outputs = model(
                    input_ids=batch['input_ids'],
                    attention_mask=batch['attention_mask'],
                    labels=batch['labels']
                )

            val_loss += outputs.loss.item()

            # Собираем предсказания и метки для метрик
            preds = torch.argmax(outputs.logits, dim=2)
            mask = batch['labels'] != -100  # Игнорируем метки -100

            all_preds.extend(preds[mask].cpu().numpy())
            all_labels.extend(batch['labels'][mask].cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)

        # Выводим метрики
        print(f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
        if len(all_labels) > 0:
            print(classification_report(
                all_labels,
                all_preds,
                target_names=list(model.config.id2label.values()),
                zero_division=0
            ))

        # Сохраняем лучшую модель
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            os.makedirs("./wikineural_ner", exist_ok=True)
            model.save_pretrained("./wikineural_ner")
            tokenizer.save_pretrained("./wikineural_ner")
            print("Saved best model!")

    return model

def predict_with_model(text: str, model, tokenizer) -> List[Tuple[str, str]]:
    """Делает предсказания с помощью модели WikiNeural."""
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(DEVICE)
    with torch.no_grad():
        outputs = model(**inputs)

    predictions = torch.argmax(outputs.logits, dim=2)
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    labels = [model.config.id2label[p.item()] for p in predictions[0]]

    return list(zip(tokens, labels))

def main(url: str):
    # 1. Извлекаем текст с веб-страницы
    text = extract_text_from_url(url)
    print(f"Извлечённый текст:\n{text[:500]}...\n")

    # 2. Аннотируем с помощью WikiNeural
    annotations = annotate_with_wikineural(text[:512])  # Ограничиваем длину для модели
    print(f"Пример аннотаций WikiNeural:\n{annotations[:5]}\n")

    # 3. Загружаем модель и токенизатор
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME).to(DEVICE)

    # 4. Обучаем модель (дообучение)
    model = train_model(model, tokenizer, text[:2000], annotations, epochs=3)

    # 5. Делаем предсказания
    predictions = predict_with_model(text[:512], model, tokenizer)

    # 6. Выводим результаты
    print("Предсказания модели WikiNeural (первые 20 токенов):")
    for token, label in predictions[:20]:
        print(f"{token} -> {label}")

if __name__ == "__main__":
    url = "https://amulex.ru/daily/news/vozrozhdenie-lyutovolkov-iz-igry-prestolov-nauchnyj-proryv-bjfj4kf/?ysclid=mae5l3wify323318294"  # Пример URL
    main(url)

Извлечённый текст:
Блог / Новости Возрождение лютоволков из «Игры престолов»: научный прорыв Полина Недашковская 07 мая, 2025 • 2 мин • 150 Копировать ссылку Telegram VK WhatsApp Биотехнологическая компания Colossal Biosciences из Далласа совершила прорыв в области возрождения вымерших видов животных, воссоздав ужасных волков (dire wolves), известных широкой аудитории как «лютоволки» из «Игры престолов» . Процесс был долгим и сложным, но в итоге ученые смогли вернуть к жизни этот вид, вымерший более 12 тысяч лет н...



Device set to use cpu


Пример аннотаций WikiNeural:
[{'entity': 'I-MISC', 'score': np.float32(0.57293886), 'index': 8, 'word': '##гр', 'start': 43, 'end': 45}, {'entity': 'I-MISC', 'score': np.float32(0.59628475), 'index': 8, 'word': '##ы', 'start': 45, 'end': 46}, {'entity': 'I-MISC', 'score': np.float32(0.5956247), 'index': 8, 'word': 'престол', 'start': 47, 'end': 54}, {'entity': 'I-MISC', 'score': np.float32(0.5244818), 'index': 8, 'word': '##ов', 'start': 54, 'end': 56}, {'entity': 'B-PER', 'score': np.float32(0.99309933), 'index': 1, 'word': 'Пол', 'start': 74, 'end': 77}]


Epoch 1/3


Validating: 100%|██████████| 1/1 [00:01<00:00,  1.80s/it]


Train Loss: nan | Val Loss: nan

Epoch 2/3


Validating: 100%|██████████| 1/1 [00:01<00:00,  1.40s/it]


Train Loss: nan | Val Loss: nan

Epoch 3/3


Validating: 100%|██████████| 1/1 [00:01<00:00,  1.70s/it]


Train Loss: nan | Val Loss: nan
Предсказания модели WikiNeural (первые 20 токенов):
[CLS] -> O
Б -> I-MISC
##лог -> I-MISC
/ -> O
Ново -> I-MISC
##сти -> I-MISC
Во -> I-MISC
##з -> O
##ро -> O
##ждение -> I-MISC
л -> O
##ют -> O
##ово -> O
##лков -> O
из -> O
« -> I-MISC
И -> I-MISC
##гр -> I-MISC
##ы -> I-MISC
престол -> I-MISC


# STREAMLIT

In [ ]:
pip install streamlit --ignore-installed blinker

  Using cached streamlit-1.45.0-py3-none-any.whl.metadata (8.9 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached click-8.2.0-py3-none-any.whl.metadata (2.5 kB)
  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached pillow-11.2.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (8.9 kB)
  Using cached protobuf-6.30.2-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
  Using cached pyarrow-20.0.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached to

In [ ]:
 pip install streamlit transformers torch beautifulsoup4 requests

In [ ]:
import streamlit as st
import requests
from bs4 import BeautifulSoup
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os

# Настройки страницы
st.set_page_config(
    page_title="NER с Qwen",
    page_icon=":mag:",
    layout="wide"
)

# Заголовок приложения
st.title("Извлечение именованных сущностей (NER) с Qwen")
st.markdown("""
Это приложение анализирует текст с веб-страницы и извлекает именованные сущности с помощью модели Qwen.
""")

# Функция для получения текста с сайта
@st.cache_data(show_spinner=False)
def get_website_text(url):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Удаляем ненужные элементы
        for element in soup(['script', 'style', 'meta', 'link', 'nav', 'footer']):
            element.decompose()

        return ' '.join(soup.stripped_strings)
    except Exception as e:
        st.error(f"Ошибка при получении текста: {e}")
        return None

# Загрузка модели с индикатором прогресса
@st.cache_resource(show_spinner="Загрузка модели...")
def load_model():
    model_name = "sberbank-ai/ruRoberta-large" # или "Qwen/Qwen2.5-32B-Instruct"
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        tokenizer.pad_token = tokenizer.eos_token
        model = AutoModelForCausalLM.from_pretrained(model_name)
        return tokenizer, model
    except Exception as e:
        st.error(f"Ошибка загрузки модели: {e}")
        return None, None

# Основной интерфейс
def main():
    # Сайдбар с настройками
    with st.sidebar:
        st.header("Настройки")
        url = st.text_input(
            "Введите URL страницы:",
            value="https://ru.wikipedia.org/wiki/История_Google"
        )
        max_length = st.slider("Максимальная длина текста:", 100, 2000, 512)
        analyze_button = st.button("Анализировать текст")

    if analyze_button and url:
        with st.spinner("Получаем текст с сайта..."):
            text = get_website_text(url)

        if text:
            # Обрезаем текст до выбранной длины
            text = text[:max_length]

            st.subheader("Извлеченный текст")
            st.text_area("Текст", text, height=200)

            with st.spinner("Анализируем текст..."):
                tokenizer, model = load_model()

                if model:
                    # Токенизация и предсказание
                    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

                    with torch.no_grad():
                        outputs = model(**inputs)

                    # Получаем предсказания
                    predictions = torch.argmax(outputs.logits, dim=-1)
                    predicted_tokens = tokenizer.batch_decode(predictions[0])

                    # Отображаем результаты
                    st.subheader("Результаты анализа")

                    col1, col2 = st.columns(2)

                    with col1:
                        st.markdown("**Оригинальные токены:**")
                        st.write(tokenizer.tokenize(text)[:50])  # Показываем первые 50 токенов

                    with col2:
                        st.markdown("**Предсказанные токены:**")
                        st.write(predicted_tokens[:50])

                    # Дополнительная информация
                    with st.expander("Технические детали"):
                        st.write(f"Размер модели: {sum(p.numel() for p in model.parameters()):,} параметров")
                        st.write(f"Длина входного текста: {len(text)} символов")
                        st.write(f"Количество токенов: {inputs.input_ids.shape[1]}")
                else:
                    st.error("Не удалось загрузить модель")

if __name__ == "__main__":
    main()

RuntimeError: Failed to import transformers.models.auto.tokenization_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
cannot import name '_center' from 'numpy._core.umath' (/usr/local/lib/python3.11/dist-packages/numpy/_core/umath.py)

In [ ]:
!streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.188.45.237:8501

https://amulex.ru/daily/news/vozrozhdenie-lyutovolkov-iz-igry-prestolov-nauchnyj-proryv-bjfj4kf/?ysclid=mae5l3wify323318294
http://localhost:8501
  Stopping...


# FASTAPI

In [ ]:
pip install fastapi uvicorn requests beautifulsoup4 torch transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.7 MB/s eta 0:00:00


In [ ]:
pip install pyngrok

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from typing import List, Dict, Optional
import requests
from bs4 import BeautifulSoup
import uvicorn
from pyngrok import ngrok

app = FastAPI(title="NER with Qwen", version="1.0")

# Конфигурация
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = "./qwen_ner"  # Путь к сохраненной модели

# Загрузка модели и токенизатора
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForTokenClassification.from_pretrained(MODEL_PATH).to(DEVICE)
    print("Model loaded successfully")
except Exception as e:
    print(f"Error loading model: {e}")
    model = None
    tokenizer = None

class TextRequest(BaseModel):
    text: str
    return_entities: Optional[bool] = True

class URLRequest(BaseModel):
    url: str
    return_entities: Optional[bool] = True

class Entity(BaseModel):
    word: str
    entity: str
    start: int
    end: int

class NERResponse(BaseModel):
    tokens: List[str]
    labels: List[str]
    entities: Optional[List[Entity]] = None
    model: str = "Qwen-NER"

def extract_text_from_url(url: str) -> str:
    """Извлекает текст с веб-страницы по URL."""
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Удаляем ненужные элементы
        for element in soup(["script", "style", "nav", "footer", "head", "meta"]):
            element.decompose()

        text = soup.get_text(separator=' ', strip=True)
        return text
    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Error extracting text from URL: {str(e)}")

def predict_entities(text: str) -> NERResponse:
    """Делает предсказания NER для текста."""
    if model is None or tokenizer is None:
        raise HTTPException(status_code=503, detail="Model not loaded")

    try:
        # Токенизация и предсказание
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(DEVICE)
        with torch.no_grad():
            outputs = model(**inputs)

        # Обработка результатов
        predictions = torch.argmax(outputs.logits, dim=2)
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
        labels = [model.config.id2label[p.item()] for p in predictions[0]]

        # Собираем сущности
        entities = []
        current_entity = None

        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label.startswith("B-"):
                if current_entity:
                    entities.append(current_entity)
                current_entity = {
                    "word": token,
                    "entity": label[2:],
                    "start": i,
                    "end": i
                }
            elif label.startswith("I-") and current_entity and label[2:] == current_entity["entity"]:
                current_entity["word"] += " " + token
                current_entity["end"] = i
            else:
                if current_entity:
                    entities.append(current_entity)
                    current_entity = None

        if current_entity:
            entities.append(current_entity)

        # Преобразуем сущности в формат с позициями в тексте
        char_pos = 0
        text_entities = []
        offset_mapping = inputs["offset_mapping"][0].cpu().numpy()

        for entity in entities:
            start_token, end_token = entity["start"], entity["end"]
            start_pos = offset_mapping[start_token][0].item()
            end_pos = offset_mapping[end_token][1].item()

            text_entities.append(Entity(
                word=entity["word"],
                entity=entity["entity"],
                start=start_pos,
                end=end_pos
            ))

        return NERResponse(
            tokens=tokens,
            labels=labels,
            entities=text_entities
        )
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Prediction error: {str(e)}")

@app.post("/predict/text", response_model=NERResponse)
async def predict_from_text(request: TextRequest):
    """API endpoint для обработки текста."""
    result = predict_entities(request.text)
    if not request.return_entities:
        result.entities = None
    return result

@app.post("/predict/url", response_model=NERResponse)
async def predict_from_url(request: URLRequest):
    """API endpoint для обработки URL."""
    text = extract_text_from_url(request.url)
    result = predict_entities(text)
    if not request.return_entities:
        result.entities = None
    return result

@app.get("/health")
async def health_check():
    """Проверка статуса сервиса."""
    return {
        "status": "OK" if model and tokenizer else "Error",
        "device": str(DEVICE),
        "model_loaded": bool(model)
    }

if __name__ == "__main__":
    uvicorn.run("__main__:app", host="0.0.0.0", port=8000, reload=True)

Error loading model: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: './qwen_ner'.


INFO:     Will watch for changes in these directories: ['/content']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [1901] using StatReload
INFO:     Stopping reloader process [1901]


In [ ]:
pip install "fastapi[standard]"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 25.7 MB/s eta 0:00:00


In [ ]:
# примеры запросов, обработка текста
!curl -X POST "http://localhost:8000/predict/url" \
-H "Content-Type: application/json" \
-d '{"url": "https://ru.wikipedia.org/wiki/История_Google", "return_entities": true}'

curl: (7) Failed to connect to localhost port 8000 after 0 ms: Connection refused
